Score the output of the CNN trained for multif0 + ismir 2015 contour classifier on the full mixes in the medleydb test set against the melody2 annotations

**This was not used in the end**

In [ ]:
import motif
import motif.plot
import numpy as np
import mir_eval
import os
import medleydb as mdb
import seaborn
import glob
import json
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Get Train/Test/Validation sets

In [ ]:
with open("../outputs/data_splits.json", 'r') as fhandle:
    dat_dict = json.load(fhandle)

def get_file_pairs(track_id_list):
    file_pairs = []
    for track_id in track_id_list:
        mtrack = mdb.MultiTrack(track_id)
        if mtrack.dataset_version != 'V1':
            continue
        npy_path = "../comparisons/multif0/algorithm_outputs/experiment11b_output/fullmix_outputs/{}_prediction.npy".format(track_id)
        if os.path.exists(npy_path):
            file_pairs.append([npy_path, mtrack.mix_path, mtrack.melody2_fpath, track_id])
    return file_pairs

file_pairs_train = get_file_pairs(dat_dict['train'])
file_pairs_validate = get_file_pairs(dat_dict['validate'])
file_pairs_test = get_file_pairs(dat_dict['test'])

ftr_bitt = motif.feature_extractors.BitteliFeatures()

### Get train/test/validate contours, features and labels

In [ ]:
def get_XY_pairs(etr, ftr, file_pairs):
    contour_list = {}
    features_list = []
    labels_list = []

    for npy_path, mix_path, annotation, track_id in file_pairs:
        print(track_id)
        ctr = etr.compute_contours(npy_path, mix_path)
        Y_train, _ = ctr.compute_labels(annotation)
        X_train = ftr.compute_all(ctr)

        features_list.append(X_train)
        labels_list.append(Y_train)
        contour_list[track_id] = ctr

    X = np.concatenate(features_list)
    Y = np.concatenate(labels_list)
    return X, Y, contour_list

In [ ]:
etr_cnn = motif.contour_extractors.DeepSal()

X_train_cnn, Y_train_cnn, train_contours_cnn = \
    get_XY_pairs(etr_cnn, ftr_bitt, file_pairs_train)
X_validate_cnn, Y_validate_cnn, validate_contours_cnn = \
    get_XY_pairs(etr_cnn, ftr_bitt, file_pairs_validate)
X_test_cnn, Y_test_cnn, test_contours_cnn = \
    get_XY_pairs(etr_cnn, ftr_bitt, file_pairs_test)

In [ ]:
X_validate_cnn, Y_validate_cnn, validate_contours_cnn = \
    get_XY_pairs(etr_cnn, ftr_bitt, file_pairs_validate)

### Train contour classifier

In [ ]:
clf_cnn = motif.contour_classifiers.RandomForest()
clf_cnn.fit(X_train_cnn, Y_train_cnn)

### Compute classification scores

In [ ]:
def score_classifier(clf, X, Y_true):
    Y_prob = clf.predict(X)
    Y_pred = (np.array(Y_prob >= clf.threshold)).astype(int)
    scores = clf.score(Y_pred, Y_true, y_prob=Y_prob)
    return scores

In [ ]:
train_scores = score_classifier(clf_cnn, X_train_cnn, Y_train_cnn)
validate_scores = score_classifier(clf_cnn, X_validate_cnn, Y_validate_cnn)
test_scores = score_classifier(clf_cnn, X_test_cnn, Y_test_cnn)

print(train_scores)
print(validate_scores)
print(test_scores)

### Get contour melody probabilities

In [ ]:
def get_contour_melprobs(ftr, clf, contours_dict):
    scores = {}

    for trackid, ctr in contours_dict.items():
        print(trackid)
        X = ftr.compute_all(ctr)
        Y = clf.predict(X)
        scores[trackid] = Y

    return scores

In [ ]:
validation_melprobs = get_contour_melprobs(
    ftr_bitt, clf_cnn, validate_contours_cnn
)
test_melprobs = get_contour_melprobs(
    ftr_bitt, clf_cnn, test_contours_cnn
)

### Get best probability threshold

In [ ]:
reload(motif)
reload(motif.contour_decoders)
reload(motif.contour_decoders.maximum)

In [ ]:
dcd = motif.contour_decoders.MaxDecoder()
dcd.dummy()

In [ ]:
thresh_vals = np.arange(0, 0.4, 0.01)
mel_accuracy = {v: [] for v in thresh_vals}

for trackid in validate_contours_cnn.keys():
    print(trackid)
    mtrack = mdb.MultiTrack(trackid)

    ctr = validate_contours_cnn[trackid]
    scores = validation_melprobs[trackid]    
    
    mel2 = mtrack.melody2_annotation
    mel2 = np.array(mel2).T
    ref_times, ref_freqs = (mel2[0], mel2[1])

    for thresh in thresh_vals:
        dcd = motif.contour_decoders.MaxDecoder(thresh=thresh)
        est_times, est_freqs = dcd.decode(ctr, scores)

        mel_scores = mir_eval.melody.evaluate(ref_times, ref_freqs, est_times, est_freqs)
        mel_accuracy[thresh].append(mel_scores['Overall Accuracy'])

In [ ]:
accuracy_vals = [np.mean(mel_accuracy[thresh]) for thresh in thresh_vals]
best_thresh_idx = np.argmax(accuracy_vals)
best_thresh = thresh_vals[best_thresh_idx]

print("Best threshold is {} with an OA of {}".format(
    best_thresh, accuracy_vals[best_thresh_idx])
)

### Score Melody Outputs on Test Set

In [ ]:
dcd = motif.contour_decoders.MaxDecoder(thresh=best_thresh)

all_mel_scores = []
for trackid in test_contours_cnn.keys():
    print(trackid)
    mtrack = mdb.MultiTrack(trackid)
    
    ctr = test_contours_cnn[trackid]
    scores = test_melprobs[trackid]

    mel2 = mtrack.melody2_annotation
    mel2 = np.array(mel2).T
    ref_times, ref_freqs = (mel2[0], mel2[1])
    est_times, est_freqs = dcd.decode(ctr, scores)

    plt.figure(figsize=(15, 7))
    plt.title(trackid)
    plt.plot(ref_times, ref_freqs, '.k', markersize=8)
    plt.plot(est_times, est_freqs, '.r', markersize=3)
    plt.show()

    mel_scores = mir_eval.melody.evaluate(ref_times, ref_freqs, est_times, est_freqs)
    all_mel_scores.append(mel_scores)

mel_scores_df = pd.DataFrame(all_mel_scores)
mel_scores_df.to_csv("../outputs/CNN_contourclf_mel2_scores.csv")

In [ ]:
mel_scores_df.describe()

### Get scores for partial mixes

In [ ]:
track_id_list = dat_dict['test']
file_pairs_test_partial = []
for track_id in track_id_list:
    mtrack = mdb.MultiTrack(track_id)
    if mtrack.dataset_version != 'V1':
        continue
    npy_path = "../comparisons/multif0/algorithm_outputs/experiment11b_output/{}_prediction.npy".format(track_id)
    if os.path.exists(npy_path) and os.path.exists(mtrack.melody2_fpath):
        file_pairs_test_partial.append([npy_path, mtrack.mix_path, mtrack.melody2_fpath, track_id])

In [ ]:
X_test_cnn_partial, Y_test_cnn_partial, test_contours_cnn_partial = \
    get_XY_pairs(etr_cnn, ftr_bitt, file_pairs_test_partial)

In [ ]:
test_melprobs_partial = get_contour_melprobs(
    ftr_bitt, clf_cnn, test_contours_cnn_partial
)

In [ ]:
dcd = motif.contour_decoders.MaxDecoder(thresh=best_thresh)

all_mel_scores = []
for trackid in test_contours_cnn_partial.keys():
    print(trackid)
    mtrack = mdb.MultiTrack(trackid)
    
    ctr = test_contours_cnn_partial[trackid]
    scores = test_melprobs_partial[trackid]

    mel2 = mtrack.melody2_annotation
    mel2 = np.array(mel2).T
    ref_times, ref_freqs = (mel2[0], mel2[1])
    est_times, est_freqs = dcd.decode(ctr, scores)

    plt.figure(figsize=(15, 7))
    plt.title(trackid)
    plt.plot(ref_times, ref_freqs, '.k', markersize=8)
    plt.plot(est_times, est_freqs, '.r', markersize=3)
    plt.show()

    mel_scores = mir_eval.melody.evaluate(ref_times, ref_freqs, est_times, est_freqs)
    all_mel_scores.append(mel_scores)

mel_scores_df_partial = pd.DataFrame(all_mel_scores)
mel_scores_df_partial.to_csv("../outputs/CNN_contourclf_mel2_scores_partial.csv")

In [ ]:
mel_scores_df_partial.describe()